In [1]:
# 基本用法
import tensorflow as tf
import numpy as np
tf.compat.v1.reset_default_graph()

# 这个axis代表什么
# 如果axis=1，那么对于输入的input_tensor来说，根据其第一个轴进行bn
# 例如对input_tensor[:,0,:,....]整体计算一个均值，是一个唯一的数字，方差也是，然后用于input_tensor[:,0,:,....]
# input_tensor[:,1,:,....]计算一个均值，是一个唯一的数字，方差也是，然后用于input_tensor[:,1,:,....]
# 注意，是整体计算均值
bn_layer=tf.keras.layers.BatchNormalization(
    axis=-1,
    epsilon=0,
    # beta_initializer=tf.zeros_initializer,
    # gamma_initializer=tf.ones_initializer,
    # scale=False,
    # moving_mean_initializer=tf.constant_initializer(0.5),
    # moving_variance_initializer=tf.constant_initializer(0.5),
    trainable=True
)

input_arr=np.reshape(np.arange(0,72),newshape=[2,3,3,4]).astype(np.float32)
training_output=bn_layer(input_arr,training=True)
print("\ntraining output")
print(training_output)

# 可以看到moving xxx被更新了 不需要像tf1那样手动update

print("\n moving xxx 被更新了")
print(bn_layer.weights)

# 可以看到这时候使用了moving_xxx做正则，结果不同了
testing_output=bn_layer(input_arr,training=False)
print("\nnot training output")
print(testing_output)


training output
tf.Tensor(
[[[[-1.6383561  -1.6383561  -1.6383561  -1.6383561 ]
   [-1.4456083  -1.4456083  -1.4456083  -1.4456083 ]
   [-1.2528605  -1.2528605  -1.2528605  -1.2528605 ]]

  [[-1.0601127  -1.0601127  -1.0601127  -1.0601127 ]
   [-0.867365   -0.867365   -0.867365   -0.867365  ]
   [-0.6746172  -0.6746172  -0.6746172  -0.6746172 ]]

  [[-0.48186943 -0.48186943 -0.48186943 -0.48186943]
   [-0.28912166 -0.28912166 -0.28912166 -0.28912166]
   [-0.09637389 -0.09637389 -0.09637389 -0.09637389]]]


 [[[ 0.09637389  0.09637389  0.09637389  0.09637389]
   [ 0.28912166  0.28912166  0.28912166  0.28912166]
   [ 0.48186943  0.48186943  0.48186943  0.48186943]]

  [[ 0.6746172   0.6746172   0.6746172   0.6746172 ]
   [ 0.867365    0.867365    0.867365    0.867365  ]
   [ 1.0601127   1.0601127   1.0601127   1.0601127 ]]

  [[ 1.2528605   1.2528605   1.2528605   1.2528605 ]
   [ 1.4456083   1.4456083   1.4456083   1.4456083 ]
   [ 1.6383561   1.6383561   1.6383561   1.6383561 ]]]], sh

bn_layer.updates


当然，这个功能仍然没有被删除，这个功能是在tf1中使用的。
WARNING:tensorflow:From <ipython-input-2-db43d979b0fc>:1: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.



momentum就是滑动平均值的decay


说个恶心事，要区分BatchNormalization求均值的方法和reduce_mean求均值的方法

BatchNormalization求均值的时候，是根据axis求整体的均值，而reduce_mean是根据axis求对应位置的均值

假如axis=1，输入维度是[a,b,c]，那bn只会求出b个数字，即[:,0,:]的整体均值、。。。、[:,b-1,:]的整体均值

但reduce_mean会求出a*c个数字，即[:,0,:]、...、[:,b-1,:]这b个向量的对应位置的均值

